# Getting the Data

In [34]:
from typing import *
from pandas_profiling import ProfileReport
import pandas as pd
from mip import *
import numpy as np

In [2]:
def get_data():
    df = pd.read_csv('all_wines_cleaned.csv').sort_values(['Avaliações'], ascending=False)
    #Incluir somente os vinhos disponíveis e com alguma pontuação, pois essas serão nossas variáveis de decisão principais
    data = df[df.Preço_Normal.notna() & df.Pontos_Total != 0].copy().reset_index(drop=True)
    data = data.drop(['Tipo_Cat', 'Preços_Cat', 'Pontuação_Cat', 'Estoque_Cat', 'Pontos_Total', 'Preço_Sócio'], axis=1)
    data.set_index('Nome', inplace=True)
    return data

In [3]:
data = get_data()

Os vinhos do catálogo possuem cerca de 15 por cento de desconto para os associados, i.e. `Preço_Normal ~ 0.85 * Preço_Sócio`

Considerando como uma aproximação que no Preço Normal incida aproximadamente 25% do valor do Preço de Custo do Vinho, i.e. 10% do Preço_Sócio.

Como o Preço Sócio não adiciona informação para o nosso problema eliminamos essa variável

In [4]:
data['Custo'] = data['Preço_Normal'] / 1.25

Em problemas de otimização devemos ter somente variáveis numéricas,i.e. reais, ordinais e booleanas. Algumas variáveis numéricas no entanto tem muitos valores nulos, como Decantação por exemplo, outras como Safra na verdade são variáveis categóricas no formato de número. Não iremos utilizá-las em nosso problema de otimização.

In [5]:
NUM: List[str] = ['Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Temperatura', 'Teor_Alcoólico', 'Potencial_Guarda']

Existem outras variáveis categóricas com baixa cardinalidade de potencialmente possam ser transformadas e utilizadas.

In [6]:
CAT: List[str] = ['Tipo', 'País', 'Puro']

In [7]:
df = data[NUM + CAT].copy()

In [8]:
df.isnull().sum()

Custo                 0
Preço_Normal          0
Pontuação             0
Avaliações            0
Temperatura           8
Teor_Alcoólico        5
Potencial_Guarda      0
Tipo                  2
País                122
Puro                  0
dtype: int64

In [9]:
df['Temperatura'] = df.Temperatura.fillna(df.Temperatura.mean())
df['Teor_Alcoólico'] = df.Temperatura.fillna(df.Temperatura.mean())
df.isnull().any()

Custo               False
Preço_Normal        False
Pontuação           False
Avaliações          False
Temperatura         False
Teor_Alcoólico      False
Potencial_Guarda    False
Tipo                 True
País                 True
Puro                False
dtype: bool

# Definição do Problema
A variável de decisão é composta pelos vinhos. O Orcamento será utilizado como _constraint_, no entanto é ajustável.

In [10]:
BUDGET: int = 1000000
VINHOS = df.index.to_list()
N = range(len(VINHOS))

## Helper Functions

In [66]:
def optimize_model(model:Model, max_gap: float=0.05, max_seconds: int=300)->Tuple[str, Dict[str, float]]:
    model.max_gap = max_gap
    status = model.optimize(max_seconds=max_seconds)
    if status == OptimizationStatus.OPTIMAL:
        solution = 'OPTIMAL'
        print('Optimal solution cost {} found'.format(model.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        solution = 'FEASIBLE'
        print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        solution = 'UNFEASIBLE'
        print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        return {v.name:v.x for v in model.vars if abs(v.x) > 1e-6}, solution
    return None, None

def safra_stats(df: pd.DataFrame)-> None:
    n = df.Quantidade.sum()
    print(f'Nº de Vinhos Distintos: {df.shape[0]}')
    print(f'Media dos Preços: {(df.Quantidade * df.Preço_Normal).sum() / n:.2f}')
    print(f'Media da Pontuação: {(df.Quantidade * df.Pontuação).sum() / n:.2f}')
    print(f'Media do nº de Avaliações: {(df.Quantidade * df.Avaliações).sum() / n:.2f}')
    print(f'Total do Orçamento Utilizado: {sum(r.Quantidade * r.Custo for r in df.itertuples()):.2f}')
    print(f'\nDistribuição da Pontuação: \n{df.Pontuação.value_counts(ascending=False)}')
    print(f'\nDistribuição dos Tipos de Vinho: \n{df.Tipo.value_counts(ascending=False)}')
    print(f'\nDistribuição dos Países: \n{df.País.value_counts(ascending=False)}')
    print(f'\nDistribuição do Potencial de Guarda: \n{df.Potencial_Guarda.value_counts(ascending=False)}')    

In [117]:
def run_model(variable: List, 
             constraints: Dict[str, Tuple[float, float]],
             sense: str = 'MAX',
             lb: int=0,
             ub: int=1000,
             is_uniform: bool=False
             ):
    """Constructs a MIP model to optimize variable subject to `constraints. Sense of Optimization defaults to MAX
    If `is_uniform` is True, it turns the main variable `vinhos` into a binary decision variable
    which if True is equivalent to one batch of `ub` 
    """
    m = Model(sense=sense)
    
    if lb == ub:
        is_uniform = True
    
    if is_uniform:
        wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in VINHOS]    
    else:
        wines = [m.add_var(name=vinho, var_type=INTEGER, lb=lb, ub=ub) for vinho in VINHOS]    
        
    ALL = set(NUM).union(CAT)    
    assert variable in ALL, f'A variável de decisão deve pertencer ao conjunto {ALL}' 
    assert set(constraints.keys()).issubset(ALL), f'As variáveis do problema devem pertencer do conjunto {ALL}'
    
    var = df[variable].to_list()
    
    #Objective Function
    m += xsum(wines[i] * var[i] for i in N)
    
    const = constraints.copy()
    
    budget = BUDGET / ub if is_uniform else BUDGET
    
    # Main constraint is budget
    custo = df['Custo'].to_list()
    m += xsum(wines[i] * custo[i] for i in N) <= budget
    m += xsum(wines[i] * custo[i] for i in N) >= budget - df.Custo.min()

    cat = set(constraints.keys()).intersection(CAT)
    num = set(constraints.keys()).intersection(NUM)

    cat = {k: constraints[k] for k in cat}
    num = {k: constraints[k] for k in num}
    
    if not all([isinstance(val, tuple) and len(val)==2 for val in num.values()]):
        raise ValueError(f'Os constraints {num.values()} devem ser tuplas com os valores (mínimo, máximo) das variáveis')    
    
    if len(cat):
        for col, col_dict in cat.items():
            if not all([isinstance(val, tuple) and len(val) == 2 for val in col_dict.values()]):
                raise ValueError(f'Os constraints {col_dict.values()} devem ser tuplas com os valores (mínimo, máximo) das variáveis')            
            
            df_dummies = pd.get_dummies(df[col])
            for k, (minimo, maximo) in col_dict.items():
                assert minimo >= 0 and maximo <= 1, f'As variáveis categóricas são proporcionais e devem estar no intervalo [0,1]'
                m += xsum(wines[i] * df_dummies[k].to_list()[i] for i in N) >= minimo * xsum(wines[i] for i in N)
                m += xsum(wines[i] * df_dummies[k].to_list()[i] for i in N) <= maximo * xsum(wines[i] for i in N)
    for i in N:        
        for key, (minimo, maximo) in num.items():
            m += wines[i] * df[key].to_list()[i] >= minimo * wines[i]
            m += wines[i] * df[key].to_list()[i] <= maximo * wines[i]
            
            
    solution, status = optimize_model(m)
    
    if solution is not None:
        multiplier = ub if is_uniform else 1
        result = data.loc[solution.keys()].copy()
        result['Quantidade'] = [v* multiplier for v in solution.values()]        
        return result, status
    
    print('!!!No result was found for the Optimization Problem with the Variable and Constraints provided!!!')
    return None, status

Maximizar o Preço sem Constraint

In [118]:
var = 'Preço_Normal'
const = {'Pontuação': (0, 5)}
resultado, status = run_model(var, const, lb=10, ub=50)
safra_stats(resultado)
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249991.6500000004 found
Nº de Vinhos Distintos: 569
Media dos Preços: 138.21
Media da Pontuação: 4.06
Media do nº de Avaliações: 14.88
Total do Orçamento Utilizado: 999993.32

Distribuição da Pontuação: 
4.0    227
4.5    131
3.5     82
5.0     63
3.0     43
2.5     12
2.0      9
1.0      1
1.5      1
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto        347
Branco       111
Espumante     79
Rosé          19
Licoroso       6
Frisante       5
Name: Tipo, dtype: int64

Distribuição dos Países: 
França            103
Espanha            76
Itália             54
Portugal           49
Brasil             41
Estados Unidos     36
Argentina          33
África do Sul      28
Uruguai            21
Austrália           4
Alemanha            1
Hungria             1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
5.0     138
4.0     128
3.0      99
6.0      55
7.0      33
8.0      32
10.0     25
2.0      18
1.0      14
9.0       9
12.0   

,Quantidade,Custo,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,
Bear Flag Red Blend 2018,10,37.552,4.0,1014.0,0,Estados Unidos,Tinto,5.0
Pedro Teixeira Colheita Selecionada Bairrada D.O.C 2018,10,32.848,4.0,389.0,0,Portugal,Tinto,7.0
Espumante Veuve D`Argent Blanc De Blancs Brut,10,34.728,4.0,360.0,0,França,Espumante,2.0
Praça dos Marqueses Escolha I.G. Beira Atlântico 2018,10,27.200,4.0,339.0,0,Portugal,Tinto,5.0
Espumante Toro Loco D.O. Cava Brut,10,42.256,4.0,284.0,0,Espanha,Espumante,3.0
...,...,...,...,...,...,...,...,...
Fantinel Borgo Tesis Doc Grave Cabernet Sauvignon Friuli 2017,50,64.848,4.0,1.0,1,Itália,Tinto,4.0
Papa Açorda Colheita Branco 2018,50,31.904,3.0,1.0,0,Portugal,Branco,3.0
Finca Constancia Selección 2016,50,66.728,5.0,1.0,0,Espanha,Tinto,10.0


Minimizar o Preço sem Constraint

In [59]:
var = 'Preço_Normal'
const = {'Pontuação': (0, 5)}
resultado, status = run_model(var, const, lb=0, ub=100000, sense='MIN')
safra_stats(resultado)
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249995.0 found
Nº de Vinhos Distintos: 1
Media dos Preços: 49.90
Media da Pontuação: 4.00
Media do nº de Avaliações: 254.00
Total do Orçamento Utilizado: 999996.00

Distribuição da Pontuação: 
4.0    1
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto    1
Name: Tipo, dtype: int64

Distribuição dos Países: 
Espanha    1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
5.0    1
Name: Potencial_Guarda, dtype: int64


,Quantidade,Custo,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,
Viña Cosos D.O. Campo de Borja Garnacha Syrah 2018,25050,39.92,4.0,254.0,0,Espanha,Tinto,5.0


Minimizar Preço com Constraint de Pontuação e Avaliações e a distribuição dos vinhos uniforme com 1000 garrafas

In [61]:
var = 'Preço_Normal'
const = {'Pontuação': (4, 5), 'Avaliações': (10, 10000)}
resultado, status = run_model(var, const, sense='MIN')
safra_stats(resultado)
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249990.45 found
Nº de Vinhos Distintos: 15
Media dos Preços: 84.65
Media da Pontuação: 4.33
Media do nº de Avaliações: 27.93
Total do Orçamento Utilizado: 999992.36

Distribuição da Pontuação: 
4.0    7
4.5    6
5.0    2
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto        11
Espumante     2
Branco        1
Rosé          1
Name: Tipo, dtype: int64

Distribuição dos Países: 
Itália           2
Portugal         2
França           1
Uruguai          1
Espanha          1
África do Sul    1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
4.0     5
5.0     4
3.0     3
6.0     1
10.0    1
7.0     1
Name: Potencial_Guarda, dtype: int64


,Quantidade,Custo,Preço_Normal,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,,
Root: 1 Reserva Heritage Red 2018,1000,33.792,42.24,4.5,56.0,0,NaN,Tinto,5.0
Palafitos de Mar Semi Sweet Rosé Cabernet Sauvignon/Syrah 2019,1000,29.080,36.35,4.0,47.0,0,NaN,Rosé,3.0
JP Azeitão Seleção do Enólogo Tinto 2018,1000,29.080,36.35,4.0,38.0,0,Portugal,Tinto,5.0
Campo Al Moro 2016,1000,29.080,36.35,4.0,37.0,0,Itália,Tinto,4.0
V9 Gran Reserva Single Vineyard Cabernet Sauvignon 2017,1000,48.848,61.06,4.5,35.0,0,NaN,Tinto,7.0
Urmeneta Cabernet Sauvignon 2018,1000,28.144,35.18,4.5,35.0,1,NaN,Tinto,4.0
Canepa Novísimo Sauvignon Blanc 2018,1000,35.680,44.60,4.0,34.0,1,NaN,Branco,3.0
Corello I.G.P. Puglia Negroamaro 2018,1000,28.144,35.18,4.5,28.0,1,Itália,Tinto,4.0
Louis Bouillot A.O.C. Crémant de Bourgogne Rosé Brut,1000,95.920,119.90,4.0,18.0,0,França,Espumante,3.0


* Minimizar Preço 
 * Pontuação entre (4,5)
 * No Mínimo 10 Avaliações
 * Distribuição Uniforme de 1000 garrafas por vinho
 * Metade dos Vinhos Puros (Somente 1 uva)

In [62]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.5, 0.5)}}
resultado, status = run_model(var, const, sense='MIN', ub=2000)
safra_stats(resultado)
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249977.75 found
Nº de Vinhos Distintos: 9
Media dos Preços: 93.97
Media da Pontuação: 4.15
Media do nº de Avaliações: 39.49
Total do Orçamento Utilizado: 999982.20

Distribuição da Pontuação: 
4.0    7
4.5    2
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto        4
Espumante    3
Branco       1
Licoroso     1
Name: Tipo, dtype: int64

Distribuição dos Países: 
França            2
Uruguai           1
Brasil            1
Espanha           1
Estados Unidos    1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
7.0    2
5.0    2
3.0    2
6.0    1
4.0    1
2.0    1
Name: Potencial_Guarda, dtype: int64


,Quantidade,Custo,Preço_Normal,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,,
Espumante Veuve D`Argent Blanc De Blancs Brut,651,34.728,43.41,4.0,360.0,0,França,Espumante,2.0
Urmeneta Reserva Carménère 2017,2000,36.608,45.76,4.0,51.0,1,NaN,Tinto,4.0
Root: 1 Sauvignon Blanc 2018,2000,36.608,45.76,4.5,20.0,1,NaN,Branco,3.0
Viñedo De Los Vientos Alcyone Tannat Dessert Wine 500 ml,2000,99.672,124.59,4.0,20.0,1,Uruguai,Licoroso,5.0
Toro Loco Reserva Barricas D.O. Utiel-Requena 2014,2000,59.200,74.00,4.5,19.0,0,Espanha,Tinto,7.0
Louis Bouillot A.O.C. Crémant de Bourgogne Rosé Brut,2000,95.920,119.90,4.0,18.0,0,França,Espumante,3.0
Carnivor Zinfandel 2017,634,89.312,111.64,4.0,17.0,1,Estados Unidos,Tinto,6.0
"Espumante Magnum Chandon Réserve Brut 1,5 L",2000,131.672,164.59,4.0,12.0,0,Brasil,Espumante,5.0
Witral Limited Edition Syrah 2016,17,81.792,102.24,4.0,10.0,1,NaN,Tinto,7.0


In [63]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.33, 0.66)}}
resultado, status = run_model(var, const, sense='MIN', ub=2000)
safra_stats(resultado)
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249983.1099999999 found
Nº de Vinhos Distintos: 6
Media dos Preços: 129.92
Media da Pontuação: 4.15
Media do nº de Avaliações: 17.13
Total do Orçamento Utilizado: 999986.49

Distribuição da Pontuação: 
4.0    4
5.0    1
4.5    1
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto        3
Espumante    2
Licoroso     1
Name: Tipo, dtype: int64

Distribuição dos Países: 
França       1
Argentina    1
Uruguai      1
Brasil       1
Espanha      1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
5.0     3
10.0    1
3.0     1
7.0     1
Name: Potencial_Guarda, dtype: int64


,Quantidade,Custo,Preço_Normal,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,,
Viñedo De Los Vientos Alcyone Tannat Dessert Wine 500 ml,2000,99.672,124.59,4.0,20.0,1,Uruguai,Licoroso,5.0
Toro Loco Reserva Barricas D.O. Utiel-Requena 2014,2000,59.200,74.00,4.5,19.0,0,Espanha,Tinto,7.0
Louis Bouillot A.O.C. Crémant de Bourgogne Rosé Brut,2000,95.920,119.90,4.0,18.0,0,França,Espumante,3.0
La Piu Belle 2013,446,402.728,503.41,5.0,18.0,0,NaN,Tinto,10.0
Altivo Vineyard Selection Cabernet Sauvignon 2017,1175,40.376,50.47,4.0,16.0,1,Argentina,Tinto,5.0
"Espumante Magnum Chandon Réserve Brut 1,5 L",2000,131.672,164.59,4.0,12.0,0,Brasil,Espumante,5.0


In [69]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.5, 1)}, 'País':{'França': (0.33, 1)}}
resultado, status = run_model(var, const, sense='MIN', lb=1, ub=25000)
if status:
    safra_stats(resultado)
    resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

!!!No result was found for the Optimization Problem with the Variable and constraints provided!!!


In [143]:
var = 'Avaliações'
const = {'Pontuação': (4,5), 
         'Avaliações': (10, 10000), 
         #'Puro':{1: (0.33, 1)}, 
         'País': {'França': (0.33, 1), 'Argentina': (0.2, 1), 'Brasil': (0.2, 1)}, #'Itália': (0.2, 1), 'Portugal': (0.2, 1)},
         #'Tipo': {'Branco': (0.2, 1), 'Tinto': (0.5, 1)},
         'Potencial_Guarda': (5, 50)}
resultado, status = run_model(var, const, sense='MAX', lb=1000, ub=1000)
if status:
    safra_stats(resultado)
resultado[NUM+CAT]

Optimal solution cost 1692.0 found
Nº de Vinhos Distintos: 10
Media dos Preços: 123.63
Media da Pontuação: 4.30
Media do nº de Avaliações: 169.20
Total do Orçamento Utilizado: 989056.00

Distribuição da Pontuação: 
4.0    5
4.5    4
5.0    1
Name: Pontuação, dtype: int64

Distribuição dos Tipos de Vinho: 
Tinto        8
Espumante    2
Name: Tipo, dtype: int64

Distribuição dos Países: 
França            4
Brasil            2
Argentina         2
Estados Unidos    1
Name: País, dtype: int64

Distribuição do Potencial de Guarda: 
5.0     7
10.0    2
6.0     1
Name: Potencial_Guarda, dtype: int64


,Custo,Preço_Normal,Pontuação,Avaliações,Temperatura,Teor_Alcoólico,Potencial_Guarda,Tipo,País,Puro
Nome,,,,,,,,,,
Bear Flag Red Blend 2018,37.552,46.94,4.0,1014.0,15.0,13.5,5.0,Tinto,Estados Unidos,0
Clos de Los Siete By Michel Rolland 2016,74.256,92.82,4.5,268.0,16.0,14.0,10.0,Tinto,Argentina,0
Espumante Chandon Réserve Brut,65.784,82.23,4.5,79.0,8.0,11.8,5.0,Espumante,Brasil,0
Enclos du Wine Hunter Bordeaux Supérieur 2018,62.968,78.71,4.5,77.0,15.0,13.5,5.0,Tinto,França,0
Chant du Coq Sélection Jean Vincent Bideau 2016,78.616,98.27,4.5,72.0,15.0,14.0,5.0,Tinto,França,1
Que Guapo Malbec Blend 2017,37.552,46.94,4.0,63.0,15.0,14.0,5.0,Tinto,Argentina,0
Pavillon Saint Pierre Réserve 2016,47.904,59.88,4.0,47.0,15.0,14.0,6.0,Tinto,França,0
Almaviva EPU 2015,399.200,499.00,5.0,31.0,16.0,14.5,10.0,Tinto,NaN,0
L'Ostal Cazes Eclipse Syrah 2017,53.552,66.94,4.0,29.0,15.0,13.0,5.0,Tinto,França,1
